<a href="https://colab.research.google.com/github/mhanauer/tempus_case_study_22/blob/main/tempus_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step: First I will clone the repository I created in Github.

In [2]:
!git clone https://github.com/mhanauer/tempus_case_study_22.git

Cloning into 'tempus_case_study_22'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


Step: Next I will install all the necesary python modules.

In [96]:
!pip install xgboost
!pip install lightgbm
!pip install scikit-optimize
!pip install skimpy
import pandas as pd
import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RepeatedKFold
import json
from pandas.core.algorithms import duplicated
from skimpy import clean_columns
from sklearn.feature_selection import VarianceThreshold
%cd tempus_case_study_22/

[Errno 2] No such file or directory: 'tempus_case_study_22/'
/content/tempus_case_study_22


Step: I will combine the biomarkers data and then read in the json patient profile file.

In [97]:
biomarkers_1 = pd.read_csv("biomarkers_1.csv")
biomarkers_2 = pd.read_csv("biomarkers_2.csv")
biomarkers_3 = pd.read_csv("biomarkers_3.csv")
biomarkers_4 = pd.read_csv("biomarkers_4.csv")
biomarkers = pd.concat([biomarkers_1,biomarkers_2, biomarkers_3, biomarkers_4])
targets = pd.read_csv("targets.csv")
with open('patient_profiles.json') as f:
   patient_profiles = json.load(f)
patient_profiles = pd.json_normalize(patient_profiles, record_path =['patient_profiles'])

Step: Identify which variables need to be dummy codied by reviewing the counts for factor variables in patient profiles.

In [5]:
for c in patient_profiles.drop(columns = ["patient_id"]).columns:
    print(patient_profiles[c].value_counts())

Male      1036
Female     937
MALE       202
FEMALE     176
Name: demographics.gender, dtype: int64
64.0    110
59.0     96
63.0     95
66.0     91
62.0     90
58.0     89
69.0     88
61.0     87
67.0     86
60.0     85
65.0     83
56.0     80
68.0     79
70.0     76
57.0     75
55.0     71
54.0     65
53.0     65
71.0     57
51.0     53
52.0     52
72.0     48
48.0     46
73.0     45
47.0     36
50.0     35
74.0     29
43.0     27
75.0     24
76.0     22
45.0     20
44.0     20
49.0     19
46.0     19
77.0     18
41.0     16
78.0     16
42.0     15
39.0     12
37.0     11
40.0     11
38.0     10
81.0      6
36.0      4
80.0      4
29.0      4
34.0      4
79.0      4
32.0      3
33.0      3
31.0      3
35.0      2
82.0      2
25.0      2
30.0      2
28.0      1
26.0      1
83.0      1
24.0      1
Name: demographics.age, dtype: int64
A    1562
B     349
C     216
F      82
D      72
E      70
Name: status.disease_sub_type, dtype: int64
0.0    231
1.0    154
2.0    100
Name: status.comor

Step: Lower all strings as some factors are treated differently when they are likely the same (e.g., Never, never).

Step: Then replace nan with NaN so python treats NaN correctly.  

Step: Then use clean_colums to make all variable names lower snake_case.

Step: Identify variables to be dummy coded then dummy code them.  Drop the original variables (i.e., those that were dummy coded) and then join the new dummy variables with the original (none factor variables) in patient profiles.

In [98]:
patient_profiles = patient_profiles.apply(lambda x: x.astype(str).str.lower())
patient_profiles = patient_profiles.replace('nan', np.NaN)
patient_profiles = clean_columns(patient_profiles)
dummies = patient_profiles[["demographics_gender", "status_disease_sub_type", "status_smoking_status", "demographics_race", "status_alcohol_usage", "status_exercise_frequency"]]
dummies_cleaned = pd.get_dummies(dummies)
patient_profiles = patient_profiles.drop(columns = dummies)
patient_profiles  = patient_profiles.reset_index(drop=True).join(dummies_cleaned)
patient_profiles


,patient_id,demographics_age,status_comorbidity_index,status_cohort_qualifier,status_months_since_diagnosis,status_days_since_diagnosis,status_bmi_level,demographics_gender_female,demographics_gender_male,status_disease_sub_type_a,...,demographics_race_black or african american,demographics_race_native hawaiian or other pacific islander,demographics_race_white,status_alcohol_usage_high,status_alcohol_usage_low,status_alcohol_usage_moderate,status_alcohol_usage_none,status_exercise_frequency_0.0,status_exercise_frequency_1.0,status_exercise_frequency_2.0
0,102bb8fae,68.0,0.0,true,1.0,NaN,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,10e32947f,66.0,NaN,true,0.0,NaN,NaN,1,0,1,...,0,0,1,0,0,0,0,0,0,0
2,11156e14a,61.0,1.0,true,16.0,NaN,NaN,0,1,1,...,0,0,1,0,0,0,0,0,0,0
3,113d8066d,62.0,NaN,true,0.0,NaN,NaN,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,113ec3f1,59.0,NaN,true,9.0,NaN,NaN,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,f9535f0b,70.0,NaN,true,0.0,NaN,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2347,fa0fa338,77.0,NaN,true,0.0,NaN,NaN,0,1,1,...,1,0,0,0,0,0,0,0,0,0
2348,fdb519f4,68.0,NaN,true,3.0,NaN,NaN,1,0,1,...,0,0,1,0,0,0,0,0,0,0
2349,fdd3b1c7,56.0,NaN,true,16.0,NaN,NaN,0,1,0,...,0,0,1,0,0,0,0,0,0,0


Step: Remove low variance predictors in the biomarker data set.  Defining low variance as less then 10% of the same factor (e.g., 95% 1's and 5% 0's).

In [99]:
def variance_threshold_selector(data, threshold=0.1):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]
biomarker_reduced_features = variance_threshold_selector(biomarkers.drop(columns = ["biomarker_id"]))
biomarkers =  pd.concat([biomarkers["biomarker_id"], biomarker_reduced_features], axis = 1)
biomarkers.rename(columns = {0:'biomarker_id'}, inplace = True)
biomarkers


,biomarker_id,BM00000,BM00001,BM00002,BM00003,BM00004,BM00006,BM00009,BM00011,BM00012,...,BM15138,BM15139,BM15140,BM15141,BM15144,BM15146,BM15149,BM15150,BM15151,BM15152
0,100505de2,1,1,1,0,0,1,1,0,1,...,0,0,0,0,0,1,1,1,0,0
1,10075c5c2,1,1,1,0,0,1,0,1,1,...,0,1,1,0,0,1,0,0,0,0
2,10105cb22,1,1,1,0,0,1,1,0,1,...,1,0,0,1,0,1,0,0,0,0
3,101219d6e,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,10135ad45,1,0,1,0,1,1,1,0,1,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,feaf2df,1,1,1,0,0,1,1,0,1,...,1,0,1,0,1,0,1,0,0,0
456,ff65d86c,1,1,1,0,1,0,1,0,1,...,1,0,0,0,0,1,1,0,0,0
457,ff9361bb,1,1,1,0,1,1,0,1,1,...,0,0,1,1,0,0,1,0,0,0
458,ffaae762,0,1,1,0,1,1,0,0,1,...,0,1,0,0,0,1,1,0,1,0


Step: Merging the target data with the patient profiles to get the biomarker ID so that biomarker ID data can be combined with patient profile.  I am doing inner joins and assuming the we need both the patient id and the biomarker id to provide useful information to the patient.  I would want to investigate why each participant does not have a unqiue biomarker id.

In [100]:
all_data = pd.merge(targets, patient_profiles,on='patient_id')
all_data = pd.merge(all_data, biomarkers,on='biomarker_id')
all_data  = clean_columns(all_data)
all_data

,patient_id,biomarker_id,target_label,demographics_age,status_comorbidity_index,status_cohort_qualifier,status_months_since_diagnosis,status_days_since_diagnosis,status_bmi_level,demographics_gender_female,...,bm_15138,bm_15139,bm_15140,bm_15141,bm_15144,bm_15146,bm_15149,bm_15150,bm_15151,bm_15152
0,1002df1d3,89c43bb4,0,46.0,NaN,true,20.0,NaN,NaN,0,...,0,0,0,0,0,1,0,1,1,0
1,1010441f,3ae31327,0,52.0,NaN,true,0.0,NaN,normal,0,...,1,0,1,0,0,0,0,0,1,0
2,101eb6af1,1d8f7bab9,0,55.0,NaN,true,0.0,NaN,NaN,1,...,1,1,0,0,0,1,1,0,0,0
3,10204394e,129ee8c6c,0,69.0,NaN,true,9.0,NaN,NaN,0,...,0,1,1,0,1,0,1,0,0,1
4,1021d329b,1b44145bd,0,52.0,NaN,true,10.0,NaN,NaN,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,f935591e,5f456337,1,60.0,NaN,true,9.0,NaN,NaN,0,...,0,0,1,0,0,1,0,1,0,1
1730,fa4be3df,1b4e0b08f,1,56.0,NaN,true,31.0,NaN,NaN,1,...,0,0,1,0,0,1,1,0,1,0
1731,fd416b4b,c8c12c4a,1,65.0,NaN,true,4.0,NaN,NaN,1,...,1,1,0,0,0,0,1,0,0,0
1732,fdaeb5f1,222e21be2,1,66.0,NaN,true,10.0,NaN,NaN,1,...,0,0,1,0,0,1,1,1,0,0


Step: Check the percentage of missing data.  Based on research I have found if more than 50% in a predictor is missing, we should remove it.

In [103]:
threshold = len(all_data)/2
all_data = all_data.dropna(axis=1, thresh = threshold)
all_data

,patient_id,biomarker_id,target_label,demographics_age,status_cohort_qualifier,status_months_since_diagnosis,demographics_gender_female,demographics_gender_male,status_disease_sub_type_a,status_disease_sub_type_b,...,bm_15138,bm_15139,bm_15140,bm_15141,bm_15144,bm_15146,bm_15149,bm_15150,bm_15151,bm_15152
0,1002df1d3,89c43bb4,0,46.0,true,20.0,0,1,1,0,...,0,0,0,0,0,1,0,1,1,0
1,1010441f,3ae31327,0,52.0,true,0.0,0,1,1,0,...,1,0,1,0,0,0,0,0,1,0
2,101eb6af1,1d8f7bab9,0,55.0,true,0.0,1,0,0,1,...,1,1,0,0,0,1,1,0,0,0
3,10204394e,129ee8c6c,0,69.0,true,9.0,0,1,1,0,...,0,1,1,0,1,0,1,0,0,1
4,1021d329b,1b44145bd,0,52.0,true,10.0,0,1,1,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,f935591e,5f456337,1,60.0,true,9.0,0,1,1,0,...,0,0,1,0,0,1,0,1,0,1
1730,fa4be3df,1b4e0b08f,1,56.0,true,31.0,1,0,1,0,...,0,0,1,0,0,1,1,0,1,0
1731,fd416b4b,c8c12c4a,1,65.0,true,4.0,1,0,0,0,...,1,1,0,0,0,0,1,0,0,0
1732,fdaeb5f1,222e21be2,1,66.0,true,10.0,1,0,1,0,...,0,0,1,0,0,1,1,1,0,0


Step: Seperate the outcome and predictors.  Remove the ID variables from the predictors.  Identify the correlation and remove highly correlated variables.

In [143]:
x = all_data.drop(columns = ["target_label", "patient_id", "biomarker_id"])
y = all_data.target_label
corr_matrix = x.corr('spearman').abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x = x.drop(x[to_drop], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


Step: Change demographics_age, status_cohort_qualifier, and status_months_since_diagnosis to float and remove status_cohort_qualifier no variation.

In [154]:
x[["demographics_age", "status_months_since_diagnosis"]] = x[["demographics_age", "status_months_since_diagnosis"]].apply(pd.to_numeric)
x = x.drop(columns = ["status_cohort_qualifier"])
x

,demographics_age,status_months_since_diagnosis,demographics_gender_female,status_disease_sub_type_a,status_disease_sub_type_b,status_disease_sub_type_c,status_disease_sub_type_d,status_disease_sub_type_e,status_disease_sub_type_f,status_smoking_status_current,...,bm_15138,bm_15139,bm_15140,bm_15141,bm_15144,bm_15146,bm_15149,bm_15150,bm_15151,bm_15152
0,46.0,20.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
1,52.0,0.0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,55.0,0.0,1,0,1,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,0
3,69.0,9.0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
4,52.0,10.0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,60.0,9.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,1
1730,56.0,31.0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
1731,65.0,4.0,1,0,0,0,0,0,1,1,...,1,1,0,0,0,0,1,0,0,0
1732,66.0,10.0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,0,0


Step: Develop training and validation (x_test) for both the predictors and the target variable. 

In [155]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.30, random_state=42)

Step: Set up the tuner Bayes tuner which uses prior information from the previous run to imporve the model on the objective function.

reg:logistic: Binary outcome.

auc: Area under the curve.

tree_method: Auto selects the method that is best generally based on the data size (e.g., exact for small and approx for large).

learning_rate: Learning rate how fast the algorithm learns.  Between zero and one.

max_depth: Max depth of cuts in tree.  Default is six and documentation gives warning about not going to high.

subsample: Percentage of training data to start the model.

colsample_bytree: Percentage of features selected.

reg_lambda: Defaults to one which means no impact and lower values impact the effect of predictors to low by but none zero.  For example, if age a had a small impact on the outcome the impact would be shrunk to zero.

scale_pos_weight: Helps with imbalance as the target variable is ~20% including the inverse (80/20) is a common approach.

In [163]:
estimator = xgb.XGBClassifier(
  objective = 'reg:logistic', 
  eval_metric = "auc", 
  tree_metod = 'auto')
 
search_spaces = {
  'learning_rate': (0.001, 1.0),
  'max_depth': (0, 20),
  'subsample': (0.1, 1.0),
  'colsample_bytree': (0.01, 1.0),
  'reg_lambda': (1e-9, 1.0), 
  'scale_pos_weight': (60/20, 90/20)
    }
 

Step: Set up the repeated cross validation.  I would normally run at least 5 cross validations and 5 repeats, but for the sake of time just did two each.

In [164]:
cv = RepeatedKFold(n_splits=2, n_repeats=2, random_state=2652124)

*Step*: Set up the bayes search

In [165]:
bayes_cv_tuner = BayesSearchCV(
  estimator = estimator, 
  search_spaces = search_spaces,
  cv = cv, 
  verbose = 0, 
  refit = True)

*Step*: Run the model and select the best estimator based on the auc.

In [166]:
result_bayes_tune = bayes_cv_tuner.fit(x_train, y_train)
best_estimator_bayes_tune = result_bayes_tune.best_estimator_


BayesSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=2, random_state=2652124),
              estimator=XGBClassifier(eval_metric='auc',
                                      objective='reg:logistic',
                                      tree_metod='auto'),
              search_spaces={'colsample_bytree': (0.01, 1.0),
                             'learning_rate': (0.001, 1.0),
                             'max_depth': (0, 20), 'reg_lambda': (1e-09, 1.0),
                             'scale_pos_weight': (3.0, 4.5),
                             'subsample': (0.1, 1.0)})

Step: Select the best model and save it.

Step: Saving and uploading the model so I don't have to run it again and receive different results.

In [172]:
import joblib
filename = 'best_estimator_bayes_tune.sav'
joblib.dump(best_estimator_bayes_tune, filename)
best_estimator_bayes_tune = load(filename) 
best_estimator_bayes_tune

XGBClassifier(colsample_bytree=0.6969792308620714, eval_metric='auc',
              learning_rate=0.001, max_depth=10, missing=nan,
              objective='reg:logistic', reg_lambda=1.0,
              scale_pos_weight=4.050833680832417, subsample=0.7141286205597641,
              tree_metod='auto')

Step: Evaluate the model on the testing data set.  I am using balanced accuracy as it can account for imbalanced data and does not give preferences to false positive or false negatives (unlike the F1 which controls for false positives).

Overall, the model predicts the target with 99% accuracy on the testing data set.  This frankly seems a little high.  I would want to increase the number of cross validations and repeats of those cross validations.  I also think we are overfitting the model with all the predictors.  I am not sure how many of those predictors we will actually have data on so I might want to explore the variable importance and use that to limit the predictors to reduce the overfitting. 

In [181]:
from sklearn.metrics import balanced_accuracy_score

def run_experiment(model):
    model.fit(x_test, y_test)
    y_pred = model.predict(x_test)
    print("Balanced accuracy : ", balanced_accuracy_score(y_test, y_pred))

results_xgboost_tempus = run_experiment(best_estimator_bayes_tune)


Balanced accuracy :  0.9951338199513382
